## Preprocessing

In [1]:
#초기 설정및 시스템 라이브러리
import platform
import warnings

# 데이터 시각화 라이브러리
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
print(platform.system())
warnings.filterwarnings('ignore')

# 행,열,결과값 생략 없이 보기,세팅
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', None)
%matplotlib inline

# 시각화 OS별 한글폰트 설정
if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows 폰트 설정
elif platform.system() == 'Mac':
    plt.rcParams['font.family'] = 'AppleGothic'  # Mac 폰트 설정
    
print("="*60)
print("라이브러리 로드 완료!")
print("한글 폰트 설정 완료!")
print("="*60)

Windows
라이브러리 로드 완료!
한글 폰트 설정 완료!


### Load Data
* funding_round.csv
* funds.csv

In [2]:
# Load Files
# df_acqui = pd.read_csv("./data/acquisitions.csv")
# df_degree = pd.read_csv("./data/degrees.csv")
df_fr = pd.read_csv("./data/funding_rounds.csv")
df_funds = pd.read_csv("./data/funds.csv")
# df_inv = pd.read_csv("./data/investments.csv")
# df_ipo = pd.read_csv("./data/ipos.csv")
# df_ms = pd.read_csv("./data/milestones.csv")
# df_obj = pd.read_csv("./data/objects.csv")
# df_office = pd.read_csv("./data/offices.csv")
# df_ppl = pd.read_csv("./data/people.csv")
# df_rel = pd.read_csv("./data/relationships.csv")

print("="*60)
print("Dataset 로드 완료!")
print("="*60)

Dataset 로드 완료!


### Functions

In [3]:
# 간단 요약/통계 함수 
def compute_skew_kurtosis(df, col):
    """
    한 열에 대해 왜도(skewness)와 첨도(kurtosis)를 계산해서 리턴.
    """
    s = df[col].dropna()
    skew = stats.skew(s)
    kurt = stats.kurtosis(s)
    print(f"왜도skew={skew}")
    print(f"첨도kurt={kurt}")

    return skew, kurt

# 결측치 관련 함수
def null_cnt_ratio(df, col):
    """
    특정 행의 결측치 갯수와 전체에서의 비율 계산 함수
    """
    total = len(df)
    null_cnt = df[col].isnull().sum()
    ratio = (null_cnt / total) * 100

    print(f"[{col}]")
    print(f"결측 수 : {null_cnt}")
    print(f"결측 비율 : {ratio:.2f}%")

    return null_cnt, ratio

# 결측 플래그 컬럼 추가
def add_missing_flag(df, col, flag_col=None):
    """
    결측 여부 플래그 열 추가 (0/1).
    예: col='income' → income_was_missing 열 추가
    """
    out = df.copy()
    if flag_col is None:
        flag_col = f"{col}_was_missing"
    out[flag_col] = out[col].isna().astype(int)
    return out

# 중복 관련 함수
def drop_full_duplicates(df):
    """
    완전 중복 행 제거 후 index 리셋.
    """
    return df.drop_duplicates().reset_index(drop=True)

def is_duplicated(df, cols):
    """
    중복 검사 함수 
    """
    for col in cols:
        dup_exist = df[col].duplicated().any()
        dup_cnt = df[col].duplicated().sum()

        print(f"[{col}] 중복 존재: {dup_exist} (중복 개수: {dup_cnt})")
        
# 이상치 관련 함수 
def iqr_bounds(series, k=1.5):
    """
    IQR 방식 이상치 경계 계산.
    → (lo, hi, q1, q3, iqr) 리턴
    """
    q1, q3 = series.quantile([0.25, 0.75])
    iqr = q3 - q1
    lo = q1 - k*iqr
    hi = q3 + k*iqr
    return lo, hi, q1, q3, iqr


def cap_with_iqr(df, col, new_col=None, k=1.5):
    """
    IQR 기준으로 캡핑(윈저화)한 새 열 생성.
    예: income → income_cap
    """
    out = df.copy()
    if new_col is None:
        new_col = f"{col}_cap"
    lo, hi, *_ = iqr_bounds(out[col].dropna(), k=k)
    out[new_col] = out[col].clip(lo, hi)
    return out


def add_log1p(df, col, new_col=None, clip_lower=0):
    """
    오른쪽 꼬리 완화를 위한 log1p 변환 열 추가.
    예: income_cap → income_log1p
    """
    out = df.copy()
    if new_col is None:
        new_col = f"{col}_log1p"
    out[new_col] = np.log1p(out[col].clip(lower=clip_lower))
    return out

# 값이 0인 행의 갯수와 전체에서의 비율 계산 함수
def zero_cnt_ratio(df, col):
    cond = (df[col] == 0)
    zero_cnt = cond.sum()              # True 개수 세기
    ratio = cond.mean() * 100          # True 비율 * 100
    
    print(f"[{col}]")
    print("0인 개수:", zero_cnt)
    print(f"0 비율: {ratio:.2f}%")

## funding_rounds 전처리

In [4]:
# 사용하는 컬럼으로만 구성 (불필요한 컬럼 제거)
fr_cleaned = df_fr[['funding_round_id', 'object_id', 'funded_at', 'funding_round_type', 'raised_amount_usd',
                    'pre_money_valuation_usd', 'post_money_valuation_usd', 'participants', 'is_first_round', 'is_last_round']]
print("="*60)
print("불필요한 컬럼 제거 완료!")

불필요한 컬럼 제거 완료!


In [5]:
# object_id
print("="*60)
print("[object_id 컬럼명 변경 전]")
display(fr_cleaned.columns)

# 컬럼명 변경
# as-is: object_id
# to-be: fr_c_id
fr_cleaned.rename(columns={'object_id':'fr_c_id'}, inplace=True)

print("="*60)
print("[object_id 컬럼명 변경 후]")
display(fr_cleaned.columns)

[object_id 컬럼명 변경 전]


Index(['funding_round_id', 'object_id', 'funded_at', 'funding_round_type',
       'raised_amount_usd', 'pre_money_valuation_usd',
       'post_money_valuation_usd', 'participants', 'is_first_round',
       'is_last_round'],
      dtype='object')

[object_id 컬럼명 변경 후]


Index(['funding_round_id', 'fr_c_id', 'funded_at', 'funding_round_type',
       'raised_amount_usd', 'pre_money_valuation_usd',
       'post_money_valuation_usd', 'participants', 'is_first_round',
       'is_last_round'],
      dtype='object')

In [6]:
print(fr_cleaned['funded_at'].dtype)

object


In [7]:
# funded_at
print("="*60)
print("[funded_at 형변환 전]")
print(f"[funded_at] 데이터타입: {fr_cleaned['funded_at'].dtype}")

# 날짜형으로 형변환
# as-is: object
# to-be: datetime
fr_cleaned["funded_at"]   = pd.to_datetime(fr_cleaned["funded_at"],   errors="coerce")

print("="*60)
print("[funded_at 형변환 후]")
print(f"[funded_at] 데이터타입: {fr_cleaned['funded_at'].dtype}")

# 결측치 NaT대치
fr_cleaned["funded_at"] = fr_cleaned["funded_at"].fillna(pd.NaT)
print("[funded_at] 결측치 대치 완료!")


[funded_at 형변환 전]
[funded_at] 데이터타입: object
[funded_at 형변환 후]
[funded_at] 데이터타입: datetime64[ns]
[funded_at] 결측치 대치 완료!


In [8]:
fr_cleaned['funding_round_type'].value_counts()

funding_round_type
venture           15342
angel             13163
series-a           9873
series-b           4892
series-c+          4216
other              4201
private-equity     1043
crowdfunding        111
post-ipo             87
Name: count, dtype: int64

In [9]:
# funding_round_type
print("="*60)
print(f"[funding_round_type] 데이터 타입: {fr_cleaned['funding_round_type'].dtype}")

# funding_round_type 카테고리 매핑 딕셔너리
# mapping 기준 변경
fr_type_map = {
    'venture': 'seed',
    'angel': 'seed',
    'series-a': 'series-a',
    'series-b': 'series-b',
    'series-c+': 'series-c+',
    'other': 'other',
    'private-equity': 'private-equity',
    'crowdfunding': 'seed',
    'post-ipo': 'post-ipo'
}

# 컬럼 추가 (funding_round_type을 category로 형변환한 것)
# 컬럼명: cat_fr_type
fr_cleaned['cat_fr_type'] = fr_cleaned['funding_round_type'].map(fr_type_map).fillna('other')
print("="*60)
print("[cat_fr_type] 컬럼 생성 완료!")

display(fr_cleaned['cat_fr_type'].value_counts())
print(fr_cleaned['cat_fr_type'].dtype)

[funding_round_type] 데이터 타입: object
[cat_fr_type] 컬럼 생성 완료!


cat_fr_type
seed              28616
series-a           9873
series-b           4892
series-c+          4216
other              4201
private-equity     1043
post-ipo             87
Name: count, dtype: int64

object


In [10]:
# 형변환
# as-is : object
# to-be : category
fr_cleaned['cat_fr_type'] = fr_cleaned['cat_fr_type'].astype('category')
print("="*60)
print(f"[cat_fr_type] 데이터 타입: {fr_cleaned['cat_fr_type'].dtype}")
print(fr_cleaned[['funding_round_type', 'cat_fr_type']].head())

[cat_fr_type] 데이터 타입: category
  funding_round_type cat_fr_type
0           series-b    series-b
1              angel        seed
2           series-a    series-a
3           series-b    series-b
4           series-b    series-b


In [11]:
# participants

# 컬럼 추가 (participants을 log1p 로그변환 한 것)
# 컬럼명: log_participants
fr_cleaned['log_participants'] = np.log1p(fr_cleaned['participants'])
print("="*60)
print("[log_participants] 컬럼 생성 완료!")

# 내용 확인
fr_cleaned['log_participants'] = np.log1p(fr_cleaned['participants'])
print(fr_cleaned[['participants', 'log_participants']].head())


[log_participants] 컬럼 생성 완료!
   participants  log_participants
0             2          1.098612
1             2          1.098612
2             3          1.386294
3             4          1.609438
4             2          1.098612


In [12]:
df_fr.to_csv("./data/clean/clean_fr_v1.csv", encoding="utf-8", index=False)
print("="*60)
print("funding_rounds 전처리 완료 csv 추출 완료!")
print("="*60)

funding_rounds 전처리 완료 csv 추출 완료!


### funds 전처리

In [13]:
# 사용하는 컬럼으로만 구성 (불필요한 컬럼 제거)
funds_cleaned = df_funds[['fund_id', 'object_id', 'name', 'funded_at', 'raised_amount', 'raised_currency_code']]
print("="*60)
print("불필요한 컬럼 제거 완료!")

불필요한 컬럼 제거 완료!


In [14]:
# object_id
print("="*60)
print("[object_id 컬럼명 변경 전]")
display(funds_cleaned.columns)

# 컬럼명 변경
# as-is: object_id
# to-be: funds_f_id
funds_cleaned.rename(columns={'object_id':'funds_f_id'}, inplace=True)

print("="*60)
print("[object_id 컬럼명 변경 후]")
display(funds_cleaned.columns)

[object_id 컬럼명 변경 전]


Index(['fund_id', 'object_id', 'name', 'funded_at', 'raised_amount',
       'raised_currency_code'],
      dtype='object')

[object_id 컬럼명 변경 후]


Index(['fund_id', 'funds_f_id', 'name', 'funded_at', 'raised_amount',
       'raised_currency_code'],
      dtype='object')

In [15]:
funds_cleaned['name'].value_counts()

name
Fund I                       61
Fund II                      48
Fund III                     37
Fund IV                      17
Fund VI                      14
                             ..
Gilde Healthcare III Fund     1
Capital Factory IV            1
FTV IV LP                     1
GTCR FUND XI/B LP             1
Hyper Ventures Fund I         1
Name: count, Length: 1313, dtype: int64

In [16]:
# name
print("="*60)
print(f"[name] 데이터 타입: {funds_cleaned['name'].dtype}")

# 컬럼 추가 (name을 category로 형변환한 것)
# 컬럼명: cat_funds_name
funds_cleaned['cat_funds_name'] = funds_cleaned['name'].copy()
print("="*60)
print("[cat_funds_name] 컬럼 생성 완료!")

funds_cleaned['cat_funds_name'].value_counts()

[name] 데이터 타입: object
[cat_funds_name] 컬럼 생성 완료!


cat_funds_name
Fund I                       61
Fund II                      48
Fund III                     37
Fund IV                      17
Fund VI                      14
                             ..
Gilde Healthcare III Fund     1
Capital Factory IV            1
FTV IV LP                     1
GTCR FUND XI/B LP             1
Hyper Ventures Fund I         1
Name: count, Length: 1313, dtype: int64

In [17]:
# 텍스트 정규화
# 1) 앞뒤 공백 제거
funds_cleaned['cat_funds_name'] = funds_cleaned['cat_funds_name'].str.strip()

# 2) 소문자로 변환
funds_cleaned['cat_funds_name'] = funds_cleaned['cat_funds_name'].str.lower()

# 3) 이름 앞뒤의 쉼표, 점, 슬래시 정도만 정리 (내부 텍스트는 유지)
funds_cleaned['cat_funds_name'] = funds_cleaned['cat_funds_name'].str.replace(r'^[\s\.,/]+', '', regex=True)
funds_cleaned['cat_funds_name'] = funds_cleaned['cat_funds_name'].str.replace(r'[\s\.,/]+$', '', regex=True)

# 4) The 제거
funds_cleaned['cat_funds_name'] = funds_cleaned['cat_funds_name'].str.replace(r'^the\s+', '', regex=True)

# 5) 탭, 여러 칸 공백 등을 모두 ' ' 하나로
funds_cleaned['cat_funds_name'] = funds_cleaned['cat_funds_name'].str.replace(r'\s+', ' ', regex=True)

print("="*60)
print("[cat_funds_name] 텍스트 정규화 완료!")

# 형변환
# as-is : object
# to-be : category
funds_cleaned['cat_funds_name'] = funds_cleaned['cat_funds_name'].astype('category')
print(f"[cat_funds_name] 데이터 타입: {funds_cleaned['cat_funds_name'].dtype}")
print(funds_cleaned[['name', 'cat_funds_name']].head())

[cat_funds_name] 텍스트 정규화 완료!
[cat_funds_name] 데이터 타입: category
                         name              cat_funds_name
0                 Second Fund                 second fund
1  Sequoia Israel Fourth Fund  sequoia israel fourth fund
2                  Tenth fund                  tenth fund
3           New funds acquire           new funds acquire
4                  Third fund                  third fund


In [18]:
# funded_at
print("="*60)
print("[funded_at 형변환 전]")
print(f"[funded_at] 데이터타입: {funds_cleaned['funded_at'].dtype}")

# 날짜형으로 형변환
# as-is: object
# to-be: datetime
funds_cleaned["funded_at"] = pd.to_datetime(funds_cleaned["funded_at"],   errors="coerce")

print("="*60)
print("[funded_at 형변환 후]")
print(f"[funded_at] 데이터타입: {funds_cleaned['funded_at'].dtype}")

# 결측치 NaT로 대치
funds_cleaned['funded_at'] = funds_cleaned['funded_at'].fillna(pd.NaT)
print("[funded_at] 결측치 채우기 완료!")

# 파생변수 생성 (시계열 패턴 분석 위한)
print("="*60)
# funded_year : 연도별
funds_cleaned['funded_year'] = funds_cleaned['funded_at'].dt.year
print("[funded_year] 파생변수 생성 완료!")
# funded_quarter: 분기별
funds_cleaned['funded_quarter'] = funds_cleaned['funded_at'].dt.to_period('Q').astype(str)
print("[funded_quarter] 파생변수 생성 완료!")

# 내용 확인
print("="*60)
print(funds_cleaned[['funded_at', 'funded_year', 'funded_quarter']].head())

[funded_at 형변환 전]
[funded_at] 데이터타입: object
[funded_at 형변환 후]
[funded_at] 데이터타입: datetime64[ns]
[funded_at] 결측치 채우기 완료!
[funded_year] 파생변수 생성 완료!
[funded_quarter] 파생변수 생성 완료!
   funded_at  funded_year funded_quarter
0 2008-12-16       2008.0         2008Q4
1 2008-12-17       2008.0         2008Q4
2        NaT          NaN            NaT
3        NaT          NaN            NaT
4        NaT          NaN            NaT


In [20]:
funds_cleaned.to_csv("./data/clean/clean_funds_v1.csv", encoding="utf-8", index=False)
print("="*60)
print("funds 전처리 완료 csv 추출 완료!")
print("="*60)

funds 전처리 완료 csv 추출 완료!
